In [31]:
import coremltools

## Step 1 - Load the specs of existing MLModel

In [32]:
specs = coremltools.utils.load_spec('./MNIST.mlmodel')

## Step 2 - Create a NeuralNetworkBuilder with the loaded specs

In [33]:
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=specs)

## Step 3 - Apply the specs for training inputs and outputs

In [34]:
nn_spec = builder.spec

# by inspecting nn_spec, you'll see the properties of your neural network.

# we want to set the specs for training inputs and outputs the same as 
# the specs of our original model for on-device training.
nn_spec.description.trainingInput.extend([nn_spec.description.input[0]])
nn_spec.description.trainingInput[0].shortDescription = 'Example of handwritten digit'

nn_spec.description.trainingInput.extend([nn_spec.description.output[0]])
nn_spec.description.trainingInput[1].shortDescription = 'Associated true label of example image'

## Step 4 - Set the updatable layers

You can view your updatable layers with `inspect_updatable_layers()`

For this example, dense_1 and dense_2 are updatable

In [35]:
builder.make_updatable(['dense_1', 'dense_2'])

## Step 5 - Set the loss function, optimizer, and number of epochs

For `MNIST` dataset, we can use the categorical_cross_entropy loss function

In [36]:
# Loss function
builder.set_categorical_cross_entropy_loss(name='loss_layer', input='prediction', target='trueDigit')

In [37]:
# Optimizer
from coremltools.models.neural_network import SgdParams as sgd
builder.set_sgd_optimizer(sgd(lr=0.01, batch=32))

In [38]:
# number of epochs
builder.set_epochs(32)

## Step 6 - Export as a new Core ML Model

In [39]:
from coremltools.models import MLModel
updatable_mlmodel = MLModel(nn_spec)
updatable_mlmodel.save('./UpdatableMNIST.mlmodel')